# Merge datasets

In [4]:
import pandas as pd
import os

def consolidate_csv_files(datasets_folder, output_path, columns_to_drop= ['latitude', 'longitude']):
    # Load CSVs that are not called 'consolidated.csv'
    csv_files = [
        os.path.join(datasets_folder, f)
        for f in os.listdir(datasets_folder)
        if f.endswith(".csv") and f != "consolidated.csv"
    ]

    if not csv_files:
        print("No CSV files found to merge.")
    else:
        # Load all datasets into a list
        dataframes = []
        for f in csv_files:
            df = pd.read_csv(f)
            # Drop unwanted columns if they exist
            df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
            dataframes.append(df)

        # Check that all columns match
        first_cols = dataframes[0].columns.tolist()
        all_match = all(df.columns.tolist() == first_cols for df in dataframes)

        if all_match:
            print("Columns match, merging all files...")
            df_all = pd.concat(dataframes, axis=0, ignore_index=True)
            output_path = os.path.join(datasets_folder, "consolidated.csv")
            df_all.to_csv(output_path, index=False)
            print(f"Merged CSV saved as '{output_path}'")
        else:
            print("WARNING: Not all CSV files have matching columns!")
            for i, df in enumerate(dataframes):
                print(f"{csv_files[i]} columns: {df.columns.tolist()}")

In [ ]:
# Merge daily datasets
consolidate_csv_files(datasets_folder="../datasets/daily", output_path="../datasets/daily/consolidated.csv")

In [5]:
# Merge hourly datasets
consolidate_csv_files(datasets_folder="../datasets/hourly", output_path="../datasets/daily/consolidated.csv")

Columns match, merging all files...
Merged CSV saved as '../datasets/hourly\consolidated.csv'


# Dataset Preprocessing

In [10]:
import numpy as np

def preprocess_data():
    daily_data_path='../datasets/daily/consolidated.csv'
    hourly_data_path='../datasets/hourly/consolidated.csv'
    cities_path='../datasets/cities.csv'
    df_monthly = None


    print("\n" + "="*70)
    print("LOADING DATA")
    print("="*70)

    # Load cities
    df_cities = pd.read_csv(cities_path)
    df_cities.rename(columns={'city_name': 'city'}, inplace=True)
    print(f"Cities: {len(df_cities)}")

    # Load daily data
    df_daily = pd.read_csv(daily_data_path)
    df_daily.rename(columns={'city_name': 'city', 'datetime': 'date'}, inplace=True)
    print(f"Daily records: {len(df_daily)}")

    # Load hourly data
    df_hourly = pd.read_csv(hourly_data_path)
    df_hourly.rename(columns={
        'city_name': 'city', 
        'datetime': 'date',
        'relative_humidity_2m': 'humidity_2m'
    }, inplace=True)
    print(f"Hourly records: {len(df_hourly)}")

    ### PREPROCESS DATA 

    print("\n" + "="*70)
    print("PREPROCESSING DATA")
    print("="*70)

    # Convert dates
    df_daily['date'] = pd.to_datetime(df_daily['date'])
    df_hourly['date'] = pd.to_datetime(df_hourly['date'])

    # Aggregate hourly to daily
    print("Aggregating hourly to daily...")
    hourly_daily = df_hourly.groupby(['city', 'date']).agg({
        'humidity_2m': 'mean',
        'surface_pressure': 'mean'
    }).reset_index()

    # Merge with daily data
    df_merged = df_daily.merge(hourly_daily, on=['city', 'date'], how='left')

    # Aggregate to monthly
    print("Aggregating daily to monthly...")
    df_merged['year'] = df_merged['date'].dt.year
    df_merged['month'] = df_merged['date'].dt.month

    df_monthly = df_merged.groupby(['city', 'year', 'month']).agg({
        'temperature_2m_mean': 'mean',
        'humidity_2m': 'mean',
        'surface_pressure': 'mean',
        'precipitation_sum': 'sum'
    }).reset_index()

    df_monthly.rename(columns={
        'temperature_2m_mean': 'temperature',
        'humidity_2m': 'humidity',
        'surface_pressure': 'air_pressure',
        'precipitation_sum': 'monthly_rainfall'
    }, inplace=True)

    # Merge with coordinates
    df_monthly = df_monthly.merge(
        df_cities[['city', 'latitude', 'longitude']], 
        on='city', 
        how='left'
    )

    # Add cyclical time encoding for months
    print("\nAdding cyclical time encoding (sin/cos for months)...")
    df_monthly['month_sin'] = np.sin(2 * np.pi * df_monthly['month'] / 12)
    df_monthly['month_cos'] = np.cos(2 * np.pi * df_monthly['month'] / 12)

    # Sort to ensure proper temporal order
    df_monthly = df_monthly.sort_values(
        by=['city', 'year', 'month']
    )

    # Add lagged rainfall per city
    grouped = df_monthly.groupby('city')['monthly_rainfall']
    df_monthly['monthly_rainfall_lag_1'] = grouped.shift(1)

    return df_monthly


month_to_season = {
    1: 'DJF', 2: 'DJF', 3: 'JFM',
    4: 'FMA', 5: 'FMA', 6: 'AMJ',
    7: 'MJJ', 8: 'JJA', 9: 'JAS',
    10: 'ASO', 11: 'SON', 12: 'NDJ'
}

# Add ENSO indices
def add_enso_indices(df, oni_data_path='../datasets/oni_indices.csv'):
    """Add ENSO (El Niño Southern Oscillation) indices"""
    oni_data = pd.read_csv(oni_data_path, index_col='year')

    def get_oni(row):
        if row['year'] in oni_data.index:
            season_col = month_to_season.get(row['month'])
            return oni_data.at[row['year'], season_col]
        else:
            return 0
    
    df['oni_index'] = df.apply(get_oni, axis=1)
    df['el_nino'] = (df['oni_index'] > 0.5).astype(int)
    df['la_nina'] = (df['oni_index'] < -0.5).astype(int)

    return df


df_monthly = preprocess_data()
add_enso_indices(df_monthly)

df_monthly.to_csv('../datasets/monthly.csv', index=False)


LOADING DATA
Cities: 141
Daily records: 768797
Hourly records: 18451128

PREPROCESSING DATA
Aggregating hourly to daily...
Aggregating daily to monthly...

Adding cyclical time encoding (sin/cos for months)...
